# Multi-circle selection over plot

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.widgets import EllipseSelector, RectangleSelector
import matplotlib.patches as patches
from matplotlib.backend_bases import MouseButton

In [3]:
class Circle:
    """Class to handle circle creation and properties
    """    
    def __init__(self, ax, center, radius, **kwargs):
        self.ax = ax
        self.center = center
        self.radius = radius
        self.circ = patches.Circle(center, radius, **kwargs)
        self.center_crosshair = self.ax.scatter(*self.center, c='r', marker='+')
        self.ax.add_patch(self.circ)
        self.ax.figure.canvas.draw_idle()
    
    def contains_event(self, event):
        return self.circ.contains(event)[0]
    
    def remove(self):
        self.circ.remove()
        self.center_crosshair.remove()
        self.ax.figure.canvas.draw_idle()
    
    def set_offsets(self, center):
        self.center = center
        self.circ.center = center
        self.center_crosshair.set_offsets(center)
        self.ax.figure.canvas.draw_idle()
    
    def set_radius(self, radius):
        self.radius = radius
        self.circ.radius = radius
        self.ax.figure.canvas.draw_idle()

    def set_edge_style(self, linestyle, linewidth):
        self.circ.set_linestyle(linestyle)
        self.circ.set_linewidth(linewidth)
        self.ax.figure.canvas.draw_idle()
    
class CircleSelector:
    """Class to handle multi circle selections

    Returns
    -------
    _type_
        _description_
    """    
    SCROLL_STEP = 0.05
    MIN_RADIUS = 0.05
    def __init__(self, ax):
        fig.canvas.mpl_connect('button_press_event', self.on_press)
        fig.canvas.mpl_connect('scroll_event', self.on_scroll)
        fig.canvas.mpl_connect('button_release_event', self.on_release)
        fig.canvas.mpl_connect('motion_notify_event', self.on_move)
        self.axes = ax
        self.circles = []  # List to store all circles
        self.active_circle = None
        self.pressevent = None

    def get_active_circle(self, event):
        """Get the circle that contains the event, if any

        Parameters
        ----------
        event : _type_
            _description_

        Returns
        -------
        _type_
            _description_
        """        
        for circle in self.circles:
            if circle.contains_event(event):
                return circle
        return None

    def on_press(self, event):
        """Create a circle with middle click or move an existing circle with left click

        Parameters
        ----------
        event : _type_
            _description_
        """        
        if event.inaxes != self.axes:
            return
        self.clear_all_circle_selections()
        if event.button == MouseButton.MIDDLE:
            self.circles.append(Circle(self.axes, (event.xdata, event.ydata), 0.25, alpha=0.1, fc='yellow', ec='red'))
            self.active_circle = self.get_active_circle(event)
        elif event.button == MouseButton.LEFT:
            self.active_circle = self.get_active_circle(event)
            if self.active_circle is not None:
                self.pressevent = event
                # Store the initial press coordinates and the circle's initial center
                self.initial_press_xdata = event.xdata
                self.initial_press_ydata = event.ydata
                self.initial_circle_center = self.active_circle.center
                # Add contour to the circle
                self.active_circle.set_edge_style('dashed', 1)
                self.axes.figure.canvas.draw_idle()

    def clear_all_circle_selections(self):
        """Clear all circle selections"""   
        for circle in self.circles:
            circle.set_edge_style('solid', 0)
        self.axes.figure.canvas.draw_idle()

    def on_scroll(self, event):
        """Increase or decrease circle radius with scroll wheel

        Parameters
        ----------
        event : _type_
            _description_
        """
        if event.inaxes != self.axes:
            return
        if self.active_circle is None:
            return
        if self.active_circle.contains_event(event):
            increment = self.SCROLL_STEP if event.button == 'up' else -self.SCROLL_STEP
            self.active_circle.set_radius(self.active_circle.radius + increment)
            if self.active_circle.radius < self.MIN_RADIUS:
                self.active_circle.remove()
                self.circles.remove(self.active_circle)
                self.active_circle = None

    def on_move(self, event):
        """Move circle with left click

        Parameters
        ----------
        event : _type_
            _description_
        """
        if not hasattr(self, 'pressevent'):
            return
        # If there is no press event (circle was not clicked) or the mouse is not in the press event axes, return
        if self.pressevent is None or event.inaxes != self.pressevent.inaxes:
            return
        # If there is no circle drawn or the mouse is not in the circle, return
        if self.active_circle is None:
            return
        dx = event.xdata - self.initial_press_xdata
        dy = event.ydata - self.initial_press_ydata
        new_center = (self.initial_circle_center[0] + dx, self.initial_circle_center[1] + dy)
        self.active_circle.set_offsets(new_center)


    def on_release(self, event):
        """Reset press event

        Parameters
        ----------
        event : _type_
            _description_
        """        
        if self.pressevent is not None:
            self.pressevent = None
            
fig = plt.figure(layout='constrained')
axs = fig.subplots(1)

N = 1000 
x = np.linspace(0, 1, N)
y = np.random.gamma(2, size=N) + x

axs.scatter(x, y, s=0.1)

axs.axis([0, 1, 0, 1])
handler = CircleSelector(axs)
plt.show()

c:\Users\mazo260d\mambaforge\envs\napari-clusters-plotter-dev\lib\site-packages\ipykernel\eventloops.py:128: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  el.exec() if hasattr(el, 'exec') else el.exec_()


In [8]:
from nap_plot_tools import make_cat10_mod_cmap
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.path import Path


class Ellipse:
    """Class to handle fixed ellipse creation and properties
    """    
    def __init__(self, ax, center, radius_x, crosshair_color, **kwargs):
        self.ax = ax
        self.center = center
        self.radius_x = radius_x
        self.ellipse = patches.Ellipse(xy=center, width=radius_x, height=0.5*radius_x, **kwargs)
        self.center_crosshair = self.ax.scatter(*self.center, color=crosshair_color, marker='+')
        self.ax.add_patch(self.ellipse)
        self.ax.figure.canvas.draw_idle()
    
    def contains_event(self, event):
        return self.ellipse.contains(event)[0]
    
    def get_vertices(self):
        return self.ellipse.get_vertices()
    
    def get_path(self):
        return self.ellipse.get_path()
    
    def remove(self):
        self.ellipse.remove()
        self.center_crosshair.remove()
        self.ax.figure.canvas.draw_idle()
    
    def set_offsets(self, center):
        self.center = center
        self.ellipse.set_center(center)
        self.center_crosshair.set_offsets(center)
        self.ax.figure.canvas.draw_idle()
    
    def set_radius_x(self, radius_x):
        self.radius_x = radius_x
        self.ellipse.width = radius_x
        self.ellipse.height = 0.5*radius_x
        self.ax.figure.canvas.draw_idle()

    def set_edge_style(self, linestyle, linewidth):
        self.ellipse.set_linestyle(linestyle)
        self.ellipse.set_linewidth(linewidth)
        self.ax.figure.canvas.draw_idle()

    def set_edge_color(self, color):
        self.ellipse.set_edgecolor(color)
        self.center_crosshair.set_color(color)
        self.ax.figure.canvas.draw_idle()
    
class CustomEllipseSelector:
    """Class to handle multi ellipse selections

    Returns
    -------
    _type_
        _description_
    """    
    SCROLL_STEP = 0.05
    MIN_RADIUS = 0.05
    def __init__(self, ax, full_data, parent=None):
        # print(full_data.shape)
        # print(full_data)
        self.plotter = parent
        self.colormap = make_cat10_mod_cmap()
        fig.canvas.mpl_connect('button_press_event', self.on_press)
        fig.canvas.mpl_connect('scroll_event', self.on_scroll)
        fig.canvas.mpl_connect('button_release_event', self.on_release)
        fig.canvas.mpl_connect('motion_notify_event', self.on_move)
        self.axes = ax
        self.ellipses = []  # List to store all ellipses
        self.active_ellipse = None
        self.pressevent = None
        
        self.full_data = full_data


    def get_active_ellipse(self, event):
        """Get the ellipse that contains the event, if any

        Parameters
        ----------
        event : _type_
            _description_

        Returns
        -------
        _type_
            _description_
        """        
        for ellipse in self.ellipses:
            if ellipse.contains_event(event):
                return ellipse
        return None

    def on_press(self, event, color='yellow'):
        """Create a ellipse with middle click or move an existing ellipse with left click

        Parameters
        ----------
        event : _type_
            _description_
        """        
        if event.inaxes != self.axes:
            return
        self.clear_all_ellipse_selections()
        if event.button == MouseButton.MIDDLE:
            self.ellipses.append(Ellipse(self.axes, (event.xdata, event.ydata), 0.25, crosshair_color=self.colormap(self.plotter.color_idx), alpha=0.5, fc='none', ec=self.colormap(self.plotter.color_idx)))
            self.active_ellipse = self.get_active_ellipse(event)
            self.active_ellipse.set_edge_style('dashed', 3)
            path = self.active_ellipse.get_path()
            affine_transf = self.active_ellipse.ellipse.get_patch_transform()
            # Apply Affine 2D to transform path from axes coordinates to data coordinates
            path = affine_transf.transform_path(path)
            # Optionally plot path
            # patch = patches.PathPatch(path, edgecolor='red', lw=2, facecolor='none')
            # self.axes.add_patch(patch)

            self.ind_mask = path.contains_points(self.full_data)
            # print(self.ind_mask)
            # Get coordinates of points inside the ellipse and plot scatter with different color
            inside_points = self.full_data[self.ind_mask]
            # Find scatter plot in axes
            scatter = self.axes.collections[0]
            scatter_array = scatter.get_array()
            # Set color of points inside the ellipse
            new_scatter_array = self.ind_mask.astype(int)*self.plotter.color_idx
            # Overwrite scatter array with new array where new_array is non-zero
            scatter_array[self.ind_mask] = new_scatter_array[self.ind_mask]
            
            scatter.set_array(scatter_array)

            # self.ind = np.nonzero(self.ind_mask)[0]
            # print(self.ind)
            
            # self.color_idx += 1
        elif event.button == MouseButton.LEFT:
            self.active_ellipse = self.get_active_ellipse(event)
            if self.active_ellipse is not None:
                self.pressevent = event
                # Store the initial press coordinates and the ellipse's initial center
                self.initial_press_xdata = event.xdata
                self.initial_press_ydata = event.ydata
                self.initial_ellipse_center = self.active_ellipse.center
                # Add dashed contour to active ellipse
                self.active_ellipse.set_edge_style('dashed', 3)
                self.axes.figure.canvas.draw_idle()

    def clear_all_ellipse_selections(self):
        """Clear all ellipse selections"""   
        for ellipse in self.ellipses:
            ellipse.set_edge_style('solid', 2)
        self.axes.figure.canvas.draw_idle()

    def on_scroll(self, event):
        """Increase or decrease ellipse radius_x with scroll wheel

        Parameters
        ----------
        event : _type_
            _description_
        """
        if event.inaxes != self.axes:
            return
        if self.active_ellipse is None:
            return
        if self.active_ellipse.contains_event(event):
            increment = self.SCROLL_STEP if event.button == 'up' else -self.SCROLL_STEP
            self.active_ellipse.set_radius_x(self.active_ellipse.radius_x + increment)
            if self.active_ellipse.radius_x < self.MIN_RADIUS:
                self.active_ellipse.remove()
                self.ellipses.remove(self.active_ellipse)
                self.active_ellipse = None

    def on_move(self, event):
        """Move ellipse with left click

        Parameters
        ----------
        event : _type_
            _description_
        """
        if not hasattr(self, 'pressevent'):
            return
        # If there is no press event (ellipse was not clicked) or the mouse is not in the press event axes, return
        if self.pressevent is None or event.inaxes != self.pressevent.inaxes:
            return
        # If there is no ellipse drawn or the mouse is not in the ellipse, return
        if self.active_ellipse is None:
            return
        dx = event.xdata - self.initial_press_xdata
        dy = event.ydata - self.initial_press_ydata
        new_center = (self.initial_ellipse_center[0] + dx, self.initial_ellipse_center[1] + dy)
        self.active_ellipse.set_offsets(new_center)


    def on_release(self, event):
        """Reset press event

        Parameters
        ----------
        event : _type_
            _description_
        """        
        if self.pressevent is not None:
            self.pressevent = None

class ColorNumber:
    def __init__(self):
        self.color_idx = 0
    def new_color_idx(self, value):
        self.color_idx = value
        return self.color_idx
            
fig = plt.figure(layout='constrained')
axs = fig.subplots(1)

N = 1000 
x = np.linspace(0, 1, N)
y = np.random.gamma(2, size=N) + x


from matplotlib.colors import ListedColormap, Normalize
my_cmap = make_cat10_mod_cmap(first_color_transparent=False)

normalizer = Normalize(vmin=0, vmax=my_cmap.N - 1)

sct = axs.scatter(x, y, s=10, cmap=my_cmap, c=np.zeros(N), norm=normalizer)

# Class to temporarily mimic the plotter object containing the color spinbox
color_number = ColorNumber()

axs.axis([0, 1, 0, 0.5])
handler = CustomEllipseSelector(axs, full_data=np.array([x, y]).T, parent=color_number)
plt.show()

In [11]:
color_number.new_color_idx(2)

2

TODO:
- Turn changing of scatter inside ellipse colors into a method (tobe used everytime the ellipse is drawn, moved, increased or deleted)
- Consider making Ellipse class derived from MPL Ellipse (to inherith all methods)

In [50]:
np.array([x, y]).T

array([[0.00000000e+00, 1.08075025e+00],
       [1.00100100e-03, 1.54929075e+00],
       [2.00200200e-03, 2.45170019e+00],
       ...,
       [9.97997998e-01, 4.28117683e+00],
       [9.98998999e-01, 1.92857693e+00],
       [1.00000000e+00, 2.27486708e+00]])

In [40]:
from matplotlib.colors import ListedColormap, Normalize
my_cmap = make_cat10_mod_cmap(first_color_transparent=False)
# Get length of my_cmap
my_cmap.N

256